In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.01
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_star/2000shot/self' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.2
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'star/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
# optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    
    teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.001000
Train | 20/20 | Loss:1.0848 | MainLoss:1.0848 | Alpha:0.0253 | SPLoss:45.2560 | CLSLoss:4.7257 | top1:50.0750 | AUROC:0.0000
Test | 153/20 | Loss:0.6969 | MainLoss:0.6969 | SPLoss:97.0363 | CLSLoss:4.4243 | top1:50.1900 | AUROC:0.5083
Test | 39/20 | Loss:0.5891 | MainLoss:0.5891 | SPLoss:97.0365 | CLSLoss:4.4242 | top1:63.5385 | AUROC:0.9705

Epoch: [2 | 1000] LR: 0.001300
Train | 20/20 | Loss:0.6986 | MainLoss:0.6986 | Alpha:0.3323 | SPLoss:3.1959 | CLSLoss:4.3171 | top1:50.7250 | AUROC:0.0000
Test | 153/20 | Loss:0.6972 | MainLoss:0.6972 | SPLoss:7.0607 | CLSLoss:4.2432 | top1:49.9902 | AUROC:0.5054
Test | 39/20 | Loss:0.6670 | MainLoss:0.6670 | SPLoss:7.0607 | CLSLoss:4.2432 | top1:52.0897 | AUROC:0.9085

Epoch: [3 | 1000] LR: 0.001600
Train | 20/20 | Loss:0.6963 | MainLoss:0.6963 | Alpha:0.3327 | SPLoss:0.8740 | CLSLoss:4.2115 | top1:51.8250 | AUROC:0.0000
Test | 153/20 | Loss:0.6973 | MainLoss:0.6973 | SPLoss:2.3087 | CLSLoss:4.1954 | top1:50.1343 |


Epoch: [23 | 1000] LR: 0.003999
Train | 20/20 | Loss:0.2783 | MainLoss:0.2783 | Alpha:0.4340 | SPLoss:9.6295 | CLSLoss:5.7714 | top1:87.8000 | AUROC:0.0000
Test | 153/20 | Loss:0.7274 | MainLoss:0.7274 | SPLoss:25.1570 | CLSLoss:5.8731 | top1:71.5007 | AUROC:0.7918
Test | 39/20 | Loss:0.9685 | MainLoss:0.9685 | SPLoss:25.1570 | CLSLoss:5.8731 | top1:62.9744 | AUROC:0.6812

Epoch: [24 | 1000] LR: 0.003999
Train | 20/20 | Loss:0.2516 | MainLoss:0.2516 | Alpha:0.4447 | SPLoss:7.1508 | CLSLoss:5.9875 | top1:89.3250 | AUROC:0.0000
Test | 153/20 | Loss:0.7493 | MainLoss:0.7493 | SPLoss:19.9539 | CLSLoss:5.9884 | top1:71.2942 | AUROC:0.7881
Test | 39/20 | Loss:0.9567 | MainLoss:0.9567 | SPLoss:19.9539 | CLSLoss:5.9884 | top1:63.7308 | AUROC:0.6895

Epoch: [25 | 1000] LR: 0.003998
Train | 20/20 | Loss:0.2315 | MainLoss:0.2315 | Alpha:0.4488 | SPLoss:8.9180 | CLSLoss:6.1788 | top1:89.9750 | AUROC:0.0000
Test | 153/20 | Loss:0.7998 | MainLoss:0.7998 | SPLoss:23.5652 | CLSLoss:6.2697 | top1:71.3

Train | 20/20 | Loss:0.1641 | MainLoss:0.1641 | Alpha:0.4602 | SPLoss:5.4454 | CLSLoss:6.6778 | top1:92.8000 | AUROC:0.0000
Test | 153/20 | Loss:0.8741 | MainLoss:0.8741 | SPLoss:14.5659 | CLSLoss:6.8238 | top1:72.7982 | AUROC:0.8019
Test | 39/20 | Loss:1.3866 | MainLoss:1.3866 | SPLoss:14.5660 | CLSLoss:6.8238 | top1:59.8333 | AUROC:0.6376

Epoch: [46 | 1000] LR: 0.003989
Train | 20/20 | Loss:0.1760 | MainLoss:0.1760 | Alpha:0.4598 | SPLoss:4.7410 | CLSLoss:6.9016 | top1:92.0250 | AUROC:0.0000
Test | 153/20 | Loss:0.8249 | MainLoss:0.8249 | SPLoss:12.7214 | CLSLoss:6.9593 | top1:73.5682 | AUROC:0.8127
Test | 39/20 | Loss:1.2794 | MainLoss:1.2794 | SPLoss:12.7214 | CLSLoss:6.9593 | top1:61.5385 | AUROC:0.6597

Epoch: [47 | 1000] LR: 0.003988
Train | 20/20 | Loss:0.1702 | MainLoss:0.1702 | Alpha:0.4621 | SPLoss:4.8792 | CLSLoss:7.0024 | top1:92.6000 | AUROC:0.0000
Test | 153/20 | Loss:0.8392 | MainLoss:0.8392 | SPLoss:13.0107 | CLSLoss:6.7781 | top1:72.5000 | AUROC:0.8013
Test | 39/20 |

Train | 20/20 | Loss:0.1345 | MainLoss:0.1345 | Alpha:0.4677 | SPLoss:0.0736 | CLSLoss:6.8657 | top1:94.0750 | AUROC:0.0000
Test | 153/20 | Loss:0.8218 | MainLoss:0.8218 | SPLoss:0.1771 | CLSLoss:6.8703 | top1:73.5649 | AUROC:0.8131
Test | 39/20 | Loss:1.3430 | MainLoss:1.3430 | SPLoss:0.1771 | CLSLoss:6.8703 | top1:59.8462 | AUROC:0.6381

Epoch: [68 | 1000] LR: 0.000397
Train | 20/20 | Loss:0.1233 | MainLoss:0.1233 | Alpha:0.4717 | SPLoss:0.0494 | CLSLoss:6.8888 | top1:94.2000 | AUROC:0.0000
Test | 153/20 | Loss:0.8406 | MainLoss:0.8406 | SPLoss:0.1437 | CLSLoss:6.9139 | top1:73.6566 | AUROC:0.8144
Test | 39/20 | Loss:1.3792 | MainLoss:1.3792 | SPLoss:0.1437 | CLSLoss:6.9139 | top1:60.2564 | AUROC:0.6409

Epoch: [69 | 1000] LR: 0.000397
Train | 20/20 | Loss:0.1317 | MainLoss:0.1317 | Alpha:0.4694 | SPLoss:0.0492 | CLSLoss:6.9353 | top1:94.0000 | AUROC:0.0000
Test | 153/20 | Loss:0.8557 | MainLoss:0.8557 | SPLoss:0.1389 | CLSLoss:6.9537 | top1:73.6599 | AUROC:0.8154
Test | 39/20 | Loss

Train | 20/20 | Loss:0.1236 | MainLoss:0.1236 | Alpha:0.4712 | SPLoss:0.0767 | CLSLoss:7.2691 | top1:94.6750 | AUROC:0.0000
Test | 153/20 | Loss:0.8619 | MainLoss:0.8619 | SPLoss:0.2019 | CLSLoss:7.2730 | top1:73.7058 | AUROC:0.8160
Test | 39/20 | Loss:1.4172 | MainLoss:1.4172 | SPLoss:0.2019 | CLSLoss:7.2730 | top1:60.3590 | AUROC:0.6462

Epoch: [90 | 1000] LR: 0.000394
Train | 20/20 | Loss:0.1359 | MainLoss:0.1359 | Alpha:0.4675 | SPLoss:0.1531 | CLSLoss:7.2503 | top1:94.2000 | AUROC:0.0000
Test | 153/20 | Loss:0.8214 | MainLoss:0.8214 | SPLoss:0.3173 | CLSLoss:7.2061 | top1:73.7549 | AUROC:0.8153
Test | 39/20 | Loss:1.3642 | MainLoss:1.3642 | SPLoss:0.3173 | CLSLoss:7.2061 | top1:60.1154 | AUROC:0.6441

Epoch: [91 | 1000] LR: 0.000394
Train | 20/20 | Loss:0.1270 | MainLoss:0.1270 | Alpha:0.4706 | SPLoss:0.0656 | CLSLoss:7.2308 | top1:94.5000 | AUROC:0.0000
Test | 153/20 | Loss:0.8375 | MainLoss:0.8375 | SPLoss:0.1754 | CLSLoss:7.2305 | top1:73.7680 | AUROC:0.8162
Test | 39/20 | Loss

...

Train | 20/20 | Loss:0.0998 | MainLoss:0.0998 | Alpha:0.4766 | SPLoss:0.1464 | CLSLoss:8.1064 | top1:95.2500 | AUROC:0.0000
Test | 153/20 | Loss:0.8961 | MainLoss:0.8961 | SPLoss:0.4573 | CLSLoss:8.1374 | top1:74.0039 | AUROC:0.8201
Test | 39/20 | Loss:1.5754 | MainLoss:1.5754 | SPLoss:0.4572 | CLSLoss:8.1374 | top1:59.8974 | AUROC:0.6330

Epoch: [200 | 1000] LR: 0.000366
Train | 20/20 | Loss:0.1061 | MainLoss:0.1061 | Alpha:0.4751 | SPLoss:0.1930 | CLSLoss:8.1852 | top1:95.2500 | AUROC:0.0000
Test | 153/20 | Loss:0.9242 | MainLoss:0.9242 | SPLoss:0.5119 | CLSLoss:8.2025 | top1:73.8434 | AUROC:0.8210
Test | 39/20 | Loss:1.5920 | MainLoss:1.5920 | SPLoss:0.5119 | CLSLoss:8.2025 | top1:60.0128 | AUROC:0.6332

Epoch: [201 | 1000] LR: 0.000366
Train | 20/20 | Loss:0.1246 | MainLoss:0.1246 | Alpha:0.4705 | SPLoss:0.3168 | CLSLoss:8.1586 | top1:94.5000 | AUROC:0.0000
Test | 153/20 | Loss:0.8621 | MainLoss:0.8621 | SPLoss:0.8399 | CLSLoss:8.0810 | top1:73.6402 | AUROC:0.8208
Test | 39/20 | Lo

Train | 20/20 | Loss:0.1076 | MainLoss:0.1076 | Alpha:0.4743 | SPLoss:0.3178 | CLSLoss:8.2183 | top1:95.3500 | AUROC:0.0000
Test | 153/20 | Loss:0.8754 | MainLoss:0.8754 | SPLoss:0.5872 | CLSLoss:8.2188 | top1:73.9908 | AUROC:0.8216
Test | 39/20 | Loss:1.5360 | MainLoss:1.5360 | SPLoss:0.5872 | CLSLoss:8.2188 | top1:59.2692 | AUROC:0.6283

Epoch: [222 | 1000] LR: 0.000358
Train | 20/20 | Loss:0.1025 | MainLoss:0.1025 | Alpha:0.4757 | SPLoss:0.2589 | CLSLoss:8.2258 | top1:95.2750 | AUROC:0.0000
Test | 153/20 | Loss:0.9077 | MainLoss:0.9077 | SPLoss:0.5956 | CLSLoss:8.2388 | top1:73.9450 | AUROC:0.8212
Test | 39/20 | Loss:1.5787 | MainLoss:1.5787 | SPLoss:0.5956 | CLSLoss:8.2388 | top1:58.9359 | AUROC:0.6227

Epoch: [223 | 1000] LR: 0.000358
Train | 20/20 | Loss:0.1134 | MainLoss:0.1134 | Alpha:0.4744 | SPLoss:0.2109 | CLSLoss:8.2210 | top1:95.4750 | AUROC:0.0000
Test | 153/20 | Loss:0.8389 | MainLoss:0.8389 | SPLoss:0.6845 | CLSLoss:8.1627 | top1:73.9417 | AUROC:0.8205
Test | 39/20 | Lo


Epoch: [243 | 1000] LR: 0.000350
Train | 20/20 | Loss:0.1006 | MainLoss:0.1006 | Alpha:0.4761 | SPLoss:0.2124 | CLSLoss:8.4087 | top1:95.6750 | AUROC:0.0000
Test | 153/20 | Loss:0.9255 | MainLoss:0.9255 | SPLoss:0.5816 | CLSLoss:8.4570 | top1:73.5157 | AUROC:0.8184
Test | 39/20 | Loss:1.5951 | MainLoss:1.5951 | SPLoss:0.5816 | CLSLoss:8.4570 | top1:59.5000 | AUROC:0.6307

Epoch: [244 | 1000] LR: 0.000349
Train | 20/20 | Loss:0.1192 | MainLoss:0.1192 | Alpha:0.4720 | SPLoss:0.3869 | CLSLoss:8.3858 | top1:95.3500 | AUROC:0.0000
Test | 153/20 | Loss:0.8499 | MainLoss:0.8499 | SPLoss:1.0797 | CLSLoss:8.3148 | top1:73.4011 | AUROC:0.8172
Test | 39/20 | Loss:1.4557 | MainLoss:1.4557 | SPLoss:1.0797 | CLSLoss:8.3148 | top1:59.0128 | AUROC:0.6262

Epoch: [245 | 1000] LR: 0.000349
Train | 20/20 | Loss:0.0984 | MainLoss:0.0984 | Alpha:0.4779 | SPLoss:0.1700 | CLSLoss:8.3444 | top1:95.7000 | AUROC:0.0000
Test | 153/20 | Loss:0.8764 | MainLoss:0.8764 | SPLoss:0.5224 | CLSLoss:8.3897 | top1:73.728

Train | 20/20 | Loss:0.0992 | MainLoss:0.0992 | Alpha:0.4773 | SPLoss:0.0014 | CLSLoss:8.4006 | top1:95.6750 | AUROC:0.0000
Test | 153/20 | Loss:0.8755 | MainLoss:0.8755 | SPLoss:0.0033 | CLSLoss:8.4017 | top1:73.8630 | AUROC:0.8213
Test | 39/20 | Loss:1.5094 | MainLoss:1.5094 | SPLoss:0.0033 | CLSLoss:8.4017 | top1:59.7308 | AUROC:0.6373

Epoch: [266 | 1000] LR: 0.000034
Train | 20/20 | Loss:0.1005 | MainLoss:0.1005 | Alpha:0.4766 | SPLoss:0.0037 | CLSLoss:8.4022 | top1:95.4000 | AUROC:0.0000
Test | 153/20 | Loss:0.8752 | MainLoss:0.8752 | SPLoss:0.0079 | CLSLoss:8.4016 | top1:73.8106 | AUROC:0.8209
Test | 39/20 | Loss:1.5031 | MainLoss:1.5031 | SPLoss:0.0079 | CLSLoss:8.4016 | top1:59.5897 | AUROC:0.6360

Epoch: [267 | 1000] LR: 0.000034
Train | 20/20 | Loss:0.0893 | MainLoss:0.0893 | Alpha:0.4786 | SPLoss:0.0022 | CLSLoss:8.4013 | top1:95.7750 | AUROC:0.0000
Test | 153/20 | Loss:0.8771 | MainLoss:0.8771 | SPLoss:0.0071 | CLSLoss:8.4041 | top1:73.8598 | AUROC:0.8206
Test | 39/20 | Lo

Train | 20/20 | Loss:0.0972 | MainLoss:0.0972 | Alpha:0.4772 | SPLoss:0.0040 | CLSLoss:8.4609 | top1:95.8250 | AUROC:0.0000
Test | 153/20 | Loss:0.8895 | MainLoss:0.8895 | SPLoss:0.0110 | CLSLoss:8.4605 | top1:73.8499 | AUROC:0.8204
Test | 39/20 | Loss:1.5428 | MainLoss:1.5428 | SPLoss:0.0110 | CLSLoss:8.4606 | top1:59.6410 | AUROC:0.6357

Epoch: [288 | 1000] LR: 0.000033
Train | 20/20 | Loss:0.0931 | MainLoss:0.0931 | Alpha:0.4773 | SPLoss:0.0051 | CLSLoss:8.4659 | top1:96.1000 | AUROC:0.0000
Test | 153/20 | Loss:0.8957 | MainLoss:0.8957 | SPLoss:0.0175 | CLSLoss:8.4745 | top1:73.8401 | AUROC:0.8209
Test | 39/20 | Loss:1.5329 | MainLoss:1.5329 | SPLoss:0.0175 | CLSLoss:8.4745 | top1:59.6410 | AUROC:0.6378

Epoch: [289 | 1000] LR: 0.000033
Train | 20/20 | Loss:0.0909 | MainLoss:0.0909 | Alpha:0.4780 | SPLoss:0.0034 | CLSLoss:8.4816 | top1:95.9500 | AUROC:0.0000
Test | 153/20 | Loss:0.9019 | MainLoss:0.9019 | SPLoss:0.0091 | CLSLoss:8.4888 | top1:73.8663 | AUROC:0.8203
Test | 39/20 | Lo